In [3]:
import pandas as pd
pd.set_option("max_columns", None)

In [4]:
from lxml import etree

In [14]:
tunits = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TUniteAuSolDescriptor.xml")
weapon_managers = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TWeaponManagerModuleDescriptor.xml")
module_selectors = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TModuleSelector.xml")
turrets = {
    'TTurretUnitDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TTurretUnitDescriptor.xml"),
    'TTurretInfanterieDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TTurretInfanterieDescriptor.xml"),
    'TTurretTwoAxisDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TTurretTwoAxisDescriptor.xml"),
    'TTurretBombardierDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TTurretBombardierDescriptor.xml")
}
weapon_descriptors = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TMountedWeaponDescriptor.xml")
ammunition = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TAmmunition.xml")
fuel_xml = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TFuelModuleDescriptor.xml")
movements = {
    'TMouvementHandlerLandVehicleDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TMouvementHandlerLandVehicleDescriptor.xml"),
    'TMouvementHandlerHelicopterDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TMouvementHandlerHelicopterDescriptor.xml"),
    'TMouvementHandlerAirplaneDescriptor': etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TMouvementHandlerAirplaneDescriptor.xml"),
}
damage = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TModernWarfareDamageModuleDescriptor.xml")
visibilities = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TVisibilityModuleDescriptor.xml")
experiences = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TModernWarfareExperienceModuleDescriptor.xml")
common_damage_descriptors = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TModernWarfareCommonDamageDescriptor.xml")
blindage_properties = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TBlindageProperties.xml")
armor_descriptors = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TArmorDescriptor.xml")
tunits_non_au_sol = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TUniteDescriptor.xml")
missile_movement_handlers = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TMouvementHandler_GuidedMissileDescriptor.xml")
tscanner_configurations = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TScannerConfigurationDescriptor.xml")

In [6]:
numbers_in_letters = {
    1: 'One',
    2: 'Two',
    3: 'Three',
    4: 'Four',
    5: 'Five'
}

In [65]:
from itertools import zip_longest


def serialize_unit(tunit_xml):
    id = tunit_xml.items()[0][1]
    unit = pd.Series()
    unit['UnitDescriptorID'] = id
    for tunit_element in tunit_xml:
        tag = tunit_element.tag
        # The complex case.
        if tag == "Modules":
            # Find the WeaponManager module ID.
            modules = [collectionelement.text for collectionelement in tunit_element]
            try:
                weapon_manager_text = list(filter(lambda module: "WeaponManager" in module, modules))[0]
                flag = False
            except IndexError:
                # No WeaponManager module is attached to this unit.
                # This happens when the unit is a supply unit, for example.
                # In that case, there's nothing else to see! We can just return the unit as-is at the end of this op.
                flag = True
            # The use of the flag is to avoid recursive errors being buried in the try-catch, which makes debugging difficult.
            if flag == False:
                unit['WeaponManagerModuleSelectorID'] = weapon_manager_text.split(":")[-1].split(" ")[1]
                # Recursively pass it to serialize_weapon_manager.
                unit = serialize_weapon_manager(unit)
                
            # Find and extract the Fuel module. Note that many units, e.g. infantry, don't need fuel.
            try:
                fuel_text = list(filter(lambda module: "Fuel" in module, modules))[0]
                unit['FuelModuleSelectorID'] = fuel_text.split(":")[-1].split(" ")[1]
                unit = serialize_fuel(unit)
            except:
                pass
            
            # Find and extract the Movement module.
            movement_text = list(filter(lambda module: "Mouvement" in module, modules))[0]
            movement_selector_id = movement_text.split(":")[-1].split(" ")[1]
            unit['MovementSelectorID'] = movement_selector_id
            unit = serialize_movement(unit)

            # Find and extract the Damage module.
            damage_text = list(filter(lambda module: "Damage" in module, modules))[0]
            damage_selector_id = damage_text.split(":")[-1].split(" ")[1]
            unit['DamageSelectorID'] = damage_selector_id
            unit = serialize_damage(unit)

            # Find and extract the Visibility module.
            visibility_text = list(filter(lambda module: "Visibility" in module, modules))[0]
            visibility_selector_id = visibility_text.split(":")[-1].split(" ")[1]
            unit['VisibilitySelectorID'] = visibility_selector_id
            unit = serialize_visibility(unit)

            # Find and extract the Experience module.
            experience_text = list(filter(lambda module: "Experience" in module, modules))[0]
            experience_selector_id = experience_text.split(":")[-1].split(" ")[1]
            unit['ExperienceSelectorID'] = experience_selector_id
            unit = serialize_experience(unit)
            
            # Find and extract the Vision (Scanner) module.
            vision_text = list(filter(lambda module: "Scanner" in module, modules))[0]
            vision_selector_id = vision_text.split(":")[-1].split(" ")[1]
            unit['VisionSelectorID'] = vision_selector_id
            unit = serialize_vision(unit)            
            
        # All other cases are simple.
        elif tag == "ProductionPrice":
            production_prices = [collectionelement.text for collectionelement in tunit_element]
            # e.g. production_prices = [90, 15, 15, 15, 15].
            # This structure seems to allow variation of price based on veterancy, but this feature is not used in-game.
            # In certain cases, the production price is simply "null". In that case `production_prices` will at this
            # point be an empty list (when is production price null? Well the NATO and PACT barges are null for example).
            if len(production_prices) > 0:
                actual_price = production_prices[0]
            else:
                actual_price = "null"
            unit['ProductionPrice'] = actual_price
        elif tag == "MaxDeployableAmount":
            deployable_amounts = [collectionelement.text for collectionelement in tunit_element]
            for i, vet_availability in enumerate(['RookieDeployableAmount', 'TrainedDeployableAmount', 
                                               'HardenedDeployableAmount', 'VeteranDeployableAmount',
                                               'EliteDeployableAmount']):
                unit[vet_availability] = deployable_amounts[i]
        # Ignore junk group tags
        elif tag in ["ShowInMenu", "UnitTypeTokens"]:
            pass
        else:
            val = tunit_element.text
            unit[tag] = val
    if 'CanDeploySmoke' not in unit.index:
        unit['CanDeploySmoke'] = False
    return unit


def serialize_weapon_manager(unit):
    # First we have to break through the module_selector element containing a reference to our desired weapon_manager element.
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['WeaponManagerModuleSelectorID']))
    selector_id = selector.items()[0][1]
    unit['WeaponManagerModuleSelectorID'] = selector_id  # Could come in handy later.
    weapon_manager_pointer_text = selector.find("Default").text
    weapon_manager_module_selector_id = weapon_manager_pointer_text.split(":")[-1].split(" ")[1]
    weapon_manager = weapon_managers.find("TWeaponManagerModuleDescriptor[@id='{0}']"\
                                                .format(weapon_manager_module_selector_id))
    weapon_manager_id = weapon_manager.items()[0][1]
    unit['WeaponManagerID'] = weapon_manager_id  # Store this too.
    # Now that we have the weapon_manager, we parse it.
    for wm_element in weapon_manager:
        tag = wm_element.tag
        # Complex case: a turret; in which case we regress again.
        if tag == "TurretDescriptorList":
            turret_ids = [collectionelement.text.split(":")[-1].split(" ")[1] for collectionelement in wm_element]
            turret_types = [collectionelement.text.split(":")[-1].split(" ")[-1] for collectionelement in wm_element]
            # First though store the turret IDs and types, because why not.
            for modifier, turret_id in zip_longest(['One', 'Two', 'Three'], turret_ids):
                unit['Turret{0}ID'.format(modifier)] = turret_id
            for modifier, turret_type in zip_longest(['One', 'Two', 'Three'], turret_types):
                unit['Turret{0}Type'.format(modifier)] = turret_type
            unit = serialize_turrets(unit)
        elif tag == "Salves":
            salvos = [collectionelement.text for collectionelement in wm_element]
            for i, wep_salvos in enumerate(['SalvosWeaponOne', 'SalvosWeaponTwo', 'SalvosWeaponThree']):
                unit[wep_salvos] = salvos[i]
        elif tag == "SalvoIsMainSalvo":
            salvo_mains = [collectionelement.text for collectionelement in wm_element]
            for i, wep_salvo_m in enumerate(['SalvoWeaponOneIsMain', 'SalvoWeaponTwoIsMain', 'SalvoWeaponThreeIsMain']):
                unit[wep_salvo_m] = salvo_mains[i]
        else:
            # Ignore other elements because they are redundant:
            # HasMainSalvo, ControllerName, _ShortDatabaseName
            pass
    return unit


def serialize_turrets(unit):
    for modifier, turret_id in zip(['One', 'Two', 'Three'], ['Turret{0}ID'.format(n) for n in ['One', 'Two', 'Three']]):
        # If no turret, jump back to the top of the loop.
        if not unit[turret_id]:
            continue
        # Otherwise, nab the correct turret XML.
        turret_type = unit['Turret{0}Type'.format(modifier)]
        turret_xml = turrets[turret_type]
        turret = turret_xml.find("{0}[@id='{1}']".format(turret_type, unit[turret_id]))
        # Now remember that we want to try and remove information dependency based on turret placement from the final CSV,
        # in order to flatten this unfamilair level of heirarchy.
        # In order to do that, we will replicate turret information across all weapons.
        # In order to do THAT we need to create a turret frame equally valid for all weapons on that turret.
        turret_srs = pd.Series()
        for element in turret:
            tag = element.tag
            if tag != "MountedWeaponDescriptorList":
                turret_srs[tag] = element.text
        # Now we are ready to recurse to the mounted weapon level.
        mounted_weapons = turret.find("MountedWeaponDescriptorList")
        mounted_weapon_ids = []
        for mounted_weapon in mounted_weapons:
            mounted_weapon_id = mounted_weapon.text.split(":")[-1].split(" ")[1]
            mounted_weapon_ids.append(mounted_weapon_id)
            serialize_weapon(unit, turret_srs, len(mounted_weapon_ids), mounted_weapon_id)
    return unit


def serialize_weapon(unit, turret_srs, weapon_number,  # weapon_number is the # of this wep in the unit's list of them...
                     weapon_index):  # ...whilst weapon_index is the full database index of the weapon.
    weapon_descriptor = weapon_descriptors.find("TMountedWeaponDescriptor[@id='{0}']".format(weapon_index))
    for element in weapon_descriptor:
        tag = element.tag
        # Complex case, dive in one final time...
        if tag == "Ammunition":
            tammunition_id = element.text.split(":")[-1].split(" ")[1]
            unit["Weapon{0}AmmunitionID".format(numbers_in_letters[weapon_number])] = tammunition_id
            unit = serialize_ammo(unit, tammunition_id, weapon_number, turret_srs)
        # Ignore.
        elif tag == "EffectTag":
            pass
        # Simple case, append.
        else:
            unit["Weapon{0}{1}".format(numbers_in_letters[weapon_number], tag)] = element.text
    return unit


def serialize_ammo(unit, tammunition_id, weapon_number, turret_srs):
    ammo = ammunition.find("TAmmunition[@id='{0}']".format(tammunition_id))
    # We want to pay particular attention to a case in which a unit may have a weapon doing double duty: 
    # it can both shoot and smoke. This clouds things, since the smoking rounds get their own TAmmunition instance
    # in the weapons list, knocking off our one-to-one of each weapon corresponding to its corresponding weapon,
    # numerically, on the weapon cards. 
    #
    # To account for this, let check to see that the ammunition we are examining deals suppression damage. If it does not,
    # then it must be a smoke round, and we can handle that as a special case.
    # Note that in the negative case, this requires attaching a CanDeploySmoke of False at the last step before returning at
    # the very top of the loop, in serialize_unit.
    #
    # Note that this still leaves some exceptional cases unaccounted for. Certain units with autocannons were patched to have
    # multiple copies of that cannon, as far as I can tell, in order to account for a bug where the autocannons would not fire
    # when the main cannon was enabled. I'm not sure this hack ever worked, however---it certainly doesn't work for me now---
    # and moreover, those quadruple and quintuple weapons, present on the French autocannon tanks in particular, mess up the
    # rest of the data. 
    # 
    # This will just have to be fixed post-merge.
    #
    # EDIT: Examining the output of this notebook at this time and examining the weapons assigned to various units shows that 
    # this assumption is patently absurd, as there are many, many units in the game featuring four, five, or more weapons:
    # breaking our assumptions totally. I will need to consult with the modding community to figure out how to parse weapons
    # properly, then.
    if ammo.find("SuppressDamages").text == "null":
        unit['CanDeploySmoke'] = True
        return unit
    for element in ammo:
        if element.tag == "MissileDescriptor" and element.text != "null":
            # Descend into the missile to fetch 
            missile_unite_id = element.text.split(":")[-1].split(" ")[1]
            missile_unite = tunits_non_au_sol.find("TUniteDescriptor[@id='{0}']".format(missile_unite_id))
            unit['Weapon{0}MissileClassNameForDebug'.format(numbers_in_letters[weapon_number])] = missile_unite.find("ClassNameForDebug").text
            modules = missile_unite.find("Modules")
            module_texts = [collectionelement.text for collectionelement in modules]
            missile_movement_text = list(filter(lambda module: "MouvementHandler" in module, module_texts))[0]
            missile_movement_id = missile_movement_text.split(":")[-1].split(" ")[1]
            missile_movement_module = missile_movement_handlers.find("TMouvementHandler_GuidedMissileDescriptor[@id='{0}']".format(missile_movement_id))
            unit["Weapon{0}MissileMaxSpeed".format(numbers_in_letters[weapon_number])] = missile_movement_module.find("Maxspeed").text
            unit["Weapon{0}MissileMaxAcceleration".format(numbers_in_letters[weapon_number])] = missile_movement_module.find("MaxAcceleration").text
        else:
            unit["Weapon{0}{1}".format(numbers_in_letters[weapon_number], element.tag)] = element.text
    new_index = list(unit.index) + ["Weapon{0}{1}".format(numbers_in_letters[weapon_number], ind) for ind in turret_srs.index]
    unit = pd.concat([unit, turret_srs.reindex(["Weapon{0}{1}".format(numbers_in_letters[weapon_number], ind) for ind in turret_srs.index])])
    unit = unit.reindex(new_index)
    return unit


# Next big chunk: damage.
def serialize_damage(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['DamageSelectorID']))
    damage_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    unit['DamageModuleID'] = damage_module_id
    damage_element = damage.find("TModernWarfareDamageModuleDescriptor[@id='{0}']".format(damage_module_id))
    # Assign elements.
    unit['MaxDamages'] = damage_element.find("MaxDamages").text
    unit['MaxHPForHUD'] = damage_element.find('MaxHPForHUD').text
    unit['AutoOrientation'] = damage_element.find("AutoOrientation").text
    unit['Transporter'] = damage_element.find('Transporter').text
    unit['IsTargetableAsBoat'] = damage_element.find("IsTargetableAsBoat").text
    # Descend into TModernWarfareCommonDamageDescriptor
    common_damage_id = damage_element.find("CommonDamageDescriptor").text.split(":")[-1].split(" ")[1]
    unit['CommonDamageDescriptorID'] = common_damage_id
    common_damage = common_damage_descriptors.find("TModernWarfareCommonDamageDescriptor[@id='{0}']".format(common_damage_id))
    unit['StunDamageRegen'] = common_damage.find("StunDamagesRegen").text
    unit['StunDamageToGetStunned'] = common_damage.find("StunDamagesToGetStunned").text
    unit['MaxSuppressionDamage'] = common_damage.find("MaxSuppressionDamages").text
    # Are these variables actually useful? Will need to shake that out in the post-op.
    unit['PaliersSuppressDamage'] = tuple([e.text for e in common_damage.find("PaliersSuppressDamages")])
    unit['PaliersPhysicalDamage'] = tuple([e.text for e in common_damage.find("PaliersPhysicalDamages")])
    unit['SuppressDamagesRegenRatio'] = tuple([
           "".join(e.text.split(" ")[-3:]) for e in common_damage.find("SuppressDamagesRegenRatio")
        ])
    unit['SuppressDamageRegenRatioOutOfRange'] = common_damage.find("SuppressDamagesRegenRatioOutOfRange").text
    # I don't bother to descend into SuppressDamage or PhysicalDamageEffects because this is always the same AFAIK.
    # But these are worth investigating in the values manual!
    blindage_id = common_damage.find("BlindageProperties").text.split(":")[-1].split(" ")[1]
    unit['BlindagePropertiesID'] = blindage_id
    blindage = blindage_properties.find("TBlindageProperties[@id='{0}']".format(blindage_id))
    for direction in ['Front', 'Sides', 'Rear', 'Top']:
        armor_id = blindage.find("ArmorDescriptor{0}".format(direction)).text.split(":")[-1].split(" ")[1]
        unit['ArmorDescriptor{0}ID'.format(direction)] = armor_id
        armor_element = armor_descriptors.find("TArmorDescriptor[@id='{0}']".format(armor_id))
        unit['{0}Armor'.format(direction)] = armor_element.find("BaseBlindage").text
    return unit
    

def serialize_fuel(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['FuelModuleSelectorID']))
    fuel_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    fuel_element = fuel_xml.find("TFuelModuleDescriptor[@id='{0}']".format(fuel_module_id))
    unit['FuelCapacity'] = fuel_element.find("FuelCapacity").text
    unit['FuelMoveDuration'] = fuel_element.find('FuelMoveDuration').text
    return unit


def serialize_movement(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['MovementSelectorID']))
    movement_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    movement_module_kind = selector.find("Default").text.split(":")[-1].split(" ")[-1]
    unit['MovementModuleID'] = movement_module_id
    unit['MovementType'] = movement_module_kind
    movement_element = movements[movement_module_kind].find("{0}[@id='{1}']".format(movement_module_kind, movement_module_id))
    for variable_of_interest in ['Maxspeed', 'UnitMovingType', 'FlyingAltitude', 'MinimalAltitude', 'GunMuzzleSpeed',  # plane
                                 'CyclicManoeuvrability', 'GFactorLimit', 'LateralSpeed', 'Mass', 'MaxInclination',
                                 'RotorArea', 'TorqueManoeuvrability', 'UpwardsSpeed', 'TempsDemiTour',
                                 'MaxAcceleration', 'MaxDeceleration', 'WeaponSabordAngle',  # helo
                                 'SpeedBonusOnRoad', 'VehicleSubType', 'TerrainsToIgnoreMask']:  # land
        try:
            unit[variable_of_interest] = movement_element.find(variable_of_interest).text
        except:
            unit[variable_of_interest] = None
    return unit


def serialize_visibility(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['VisibilitySelectorID']))
    visibility_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    unit['VisibilityModuleID'] = visibility_module_id
    visibility_element = visibilities.find("TVisibilityModuleDescriptor[@id='{0}']".format(visibility_module_id))
    unit['UnitStealthBonus'] = visibility_element.find("UnitStealthBonus").text
    return unit
    
    
def serialize_experience(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['ExperienceSelectorID']))
    experience_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    unit['ExperienceModuleID'] = experience_module_id
    experience_element = experiences.find("TModernWarfareExperienceModuleDescriptor[@id='{0}']".format(experience_module_id))
    unit['ExperienceGainBySecond'] = experience_element.find("ExperienceGainBySecond").text
    unit['KillExperienceBonus'] = experience_element.find("KillExperienceBonus").text
    unit['CanWinExperience'] = experience_element.find("CanWinExperience").text
    return unit


def serialize_vision(unit):
    selector = module_selectors.find("TModuleSelector[@id='{0}']".format(unit['VisionSelectorID']))
    vision_module_id = selector.find("Default").text.split(":")[-1].split(" ")[1]
    unit['VisionModuleID'] = vision_module_id
    vision_element = tscanner_configurations.find("TScannerConfigurationDescriptor[@id='{0}']".format(vision_module_id))
    unit['VisionUnitType'] = vision_element.find("UnitType").text
    unit['DetectionTBA'] = vision_element.find("DetectionTBA").text
    unit['PorteeVision'] = vision_element.find("PorteeVision").text
    unit['OpticalStrength'] = vision_element.find("OpticalStrength").text
    unit['OpticalStrengthAltitude'] = vision_element.find("OpticalStrengthAltitude").text
    # For the below cf. http://forums.eugensystems.com/viewtopic.php?f=187&t=42266&p=652844&hilit=OpticalStrength#p652844
    for element_text in [collectionelement.text for collectionelement in vision_element.find("SpecializedOpticalStrengths")]:
        val = element_text.split(":")[-1].strip()
        if "Map: 4" in element_text:
            unit['SpecializedOpticalStrengthPlanes'] = val
        elif "Map: 6" in element_text:
            unit['SpecializedOpticalStrengthShips'] = val
    for element_text in [collectionelement.text for collectionelement in vision_element.find("SpecializedDetections")]:
        val = element_text.split(":")[-1].strip()
        if "Map: 4" in element_text:
            unit['SpecializedDetectionPlanes'] = val
        elif "Map: 6" in element_text:
            unit['SpecializedDetectionShips'] = val
    unit['PorteeVisionTBA'] = vision_element.find("PorteeVisionTBA").text
    unit['OpticalStrengthAntiradar'] = vision_element.find("OpticalStrengthAntiradar").text
    return unit

In [66]:
serialize_unit(tunits.findall("TUniteAuSolDescriptor")[0])

UnitDescriptorID                                                             14672
DescriptorId                                  00000000-0000-0000-0000-0000e8030000
WeaponManagerModuleSelectorID                                                16519
WeaponManagerID                                                              27760
TurretOneID                                                                  38484
TurretTwoID                                                                  38485
TurretThreeID                                                                 None
TurretOneType                                             TTurretTwoAxisDescriptor
TurretTwoType                                             TTurretTwoAxisDescriptor
TurretThreeType                                                               None
WeaponOneAmmunitionID                                                        66544
WeaponOneDescriptorId                         00000000-0000-0000-0600-0000e9030000
Weap

In [25]:
serialize_unit(tunits.findall("TUniteAuSolDescriptor")[1])

UnitDescriptorID                                                         14673
DescriptorId                              00000000-0000-0000-0000-0000e9030000
WeaponManagerModuleSelectorID                                            16541
WeaponManagerID                                                          27780
TurretOneID                                                              38519
TurretTwoID                                                               None
TurretThreeID                                                             None
TurretOneType                                         TTurretTwoAxisDescriptor
TurretTwoType                                                             None
TurretThreeType                                                           None
WeaponOneAmmunitionID                                                    66569
WeaponOneDescriptorId                     00000000-0000-0000-0600-00008e090000
WeaponOneName                                       

In [29]:
serialize_unit(tunits.findall("TUniteAuSolDescriptor")[0])

UnitDescriptorID                                                             14672
DescriptorId                                  00000000-0000-0000-0000-0000e8030000
WeaponManagerModuleSelectorID                                                16519
WeaponManagerID                                                              27760
TurretOneID                                                                  38484
TurretTwoID                                                                  38485
TurretThreeID                                                                 None
TurretOneType                                             TTurretTwoAxisDescriptor
TurretTwoType                                             TTurretTwoAxisDescriptor
TurretThreeType                                                               None
WeaponOneAmmunitionID                                                        66541
WeaponOneDescriptorId                         00000000-0000-0000-0600-000083070000
Weap

In [45]:
dana = _

In [52]:
dana['FrontArmor']

'6'

In [51]:
dana['RearArmor']

'5'

In [61]:
serialize_unit(tunits.findall("TUniteAuSolDescriptor")[540])[['FrontArmor', 'SidesArmor', 'TopArmor', 'RearArmor']]

('1:0.2', '10:0.5', '30:1', '40:2', '50:3')
67330
66545
66545
66546


FrontArmor    12
SidesArmor     6
TopArmor       5
RearArmor      6
dtype: object

In [7]:
print(list(serialize_unit(tunits.findall("TUniteAuSolDescriptor")[0]).index))

['UnitDescriptorID', 'DescriptorId', 'WeaponManagerModuleSelectorID', 'WeaponManagerID', 'TurretOneID', 'TurretTwoID', 'TurretThreeID', 'TurretOneType', 'TurretTwoType', 'TurretThreeType', 'WeaponOneAmmunitionID', 'WeaponOneDescriptorId', 'WeaponOneName', 'WeaponOneTypeName', 'WeaponOneTypeArme', 'WeaponOneCaliber', 'WeaponOneArme', 'WeaponOneProjectileType', 'WeaponOneWeaponCursorType', 'WeaponOnePuissance', 'WeaponOneTempsEntreDeuxTirs', 'WeaponOneTempsEntreDeuxFx', 'WeaponOnePorteeMaximale', 'WeaponOnePorteeMinimale', 'WeaponOnePorteeMaximaleBateaux', 'WeaponOnePorteeMinimaleBateaux', 'WeaponOneAngleDispersion', 'WeaponOneDispersionAtMinRange', 'WeaponOneDispersionAtMaxRange', 'WeaponOneRadiusSplashPhysicalDamages', 'WeaponOnePhysicalDamages', 'WeaponOneRadiusSplashSuppressDamages', 'WeaponOneSuppressDamages', 'WeaponOneRayonPinned', 'WeaponOneTirIndirect', 'WeaponOneTirReflexe', 'WeaponOneFX_vitesse_de_depart', 'WeaponOneFX_frottement', 'WeaponOneNoiseDissimulationMalus', 'WeaponOn

In [47]:
print(list(serialize_unit(tunits.findall("TUniteAuSolDescriptor")[0]).index))

['UnitDescriptorID', 'DescriptorId', 'WeaponManagerModuleSelectorID', 'WeaponManagerID', 'TurretOneID', 'TurretTwoID', 'TurretThreeID', 'TurretOneType', 'TurretTwoType', 'TurretThreeType', 'WeaponOneAmmunitionID', 'WeaponOneDescriptorId', 'WeaponOneName', 'WeaponOneTypeName', 'WeaponOneTypeArme', 'WeaponOneCaliber', 'WeaponOneArme', 'WeaponOneProjectileType', 'WeaponOneWeaponCursorType', 'WeaponOnePuissance', 'WeaponOneTempsEntreDeuxTirs', 'WeaponOneTempsEntreDeuxFx', 'WeaponOnePorteeMaximale', 'WeaponOnePorteeMinimale', 'WeaponOnePorteeMaximaleBateaux', 'WeaponOnePorteeMinimaleBateaux', 'WeaponOneAngleDispersion', 'WeaponOneDispersionAtMinRange', 'WeaponOneDispersionAtMaxRange', 'WeaponOneRadiusSplashPhysicalDamages', 'WeaponOnePhysicalDamages', 'WeaponOneRadiusSplashSuppressDamages', 'WeaponOneSuppressDamages', 'WeaponOneRayonPinned', 'WeaponOneTirIndirect', 'WeaponOneTirReflexe', 'WeaponOneFX_vitesse_de_depart', 'WeaponOneFX_frottement', 'WeaponOneNoiseDissimulationMalus', 'WeaponOn

In [48]:
from tqdm import tqdm_notebook

In [55]:
data = []
c = 0
for unit in tqdm_notebook(tunits.findall("TUniteAuSolDescriptor")):
    data.append(
        serialize_unit(unit)
    )
    c += 1

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\site-packages\tqdm\_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\Alex\Anaconda3\envs\wargame-analysis\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [56]:
df = pd.concat(data, axis=1).T

In [57]:
df.head()

,AcknowUnitType,AliasName,ArmorDescriptorFrontID,ArmorDescriptorRearID,ArmorDescriptorSidesID,ArmorDescriptorTopID,AutoOrientation,BlindagePropertiesID,CanDeploySmoke,CanWinExperience,Category,ClassNameForDebug,CommonDamageDescriptorID,CoutEtoile,CyclicManoeuvrability,DamageModuleID,DamageSelectorID,DeathExplosionAmmo,DescriptorId,DetectionTBA,EliteDeployableAmount,ExperienceGainBySecond,ExperienceModuleID,ExperienceSelectorID,Factory,FlyingAltitude,FrontArmor,FuelCapacity,FuelModuleSelectorID,FuelMoveDuration,GFactorLimit,GunMuzzleSpeed,HardenedDeployableAmount,HitRollECMModifier,HitRollSizeModifier,IconeType,IsPrototype,IsTargetableAsBoat,Key,KillExperienceBonus,LateralSpeed,ManageUnitOrientation,Mass,MaxAcceleration,MaxDamages,MaxDeceleration,MaxHPForHUD,MaxInclination,MaxPacks,MaxSuppressionDamage,Maxspeed,MinimalAltitude,MotherCountry,MovementModuleID,MovementSelectorID,MovementType,NameInMenuToken,Nationalite,OpticalStrength,OpticalStrengthAltitude,OpticalStrengthAntiradar,PaliersPhysicalDamage,PaliersSuppressDamage,PorteeVision,PorteeVisionTBA,PositionInMenu,ProductionPrice,ProductionTime,ProductionYear,RearArmor,RookieDeployableAmount,RotorArea,SalvoWeaponOneIsMain,SalvoWeaponThreeIsMain,SalvoWeaponTwoIsMain,SalvosWeaponOne,SalvosWeaponThree,SalvosWeaponTwo,SidesArmor,SpecializedDetectionPlanes,SpecializedDetectionShips,SpecializedOpticalStrengthShips,SpeedBonusOnRoad,StickToGround,StunDamageRegen,StunDamageToGetStunned,SuppressDamageRegenRatioOutOfRange,SuppressDamagesRegenRatio,TempsDemiTour,TerrainsToIgnoreMask,TextureForInterface,TextureMotherCountryForInterface,TextureTransportForInterface,TopArmor,TorqueManoeuvrability,TrainedDeployableAmount,Transporter,TurretNoneID,TurretNoneType,TurretOneID,TurretOneType,TurretThreeID,TurretThreeType,TurretTwoID,TurretTwoType,TypeForAcknow,UnitDescriptorID,UnitMovingType,UnitStealthBonus,UpgradeRequire,UpwardsSpeed,VehicleSubType,VeteranDeployableAmount,VisibilityModuleID,VisibilitySelectorID,VisionModuleID,VisionSelectorID,VisionUnitType,VitesseCombat,WeaponFourAffecteParNombre,WeaponFourAffichageMenu,WeaponFourAffichageMunitionParSalve,WeaponFourAmmunitionID,WeaponFourAngleDispersion,WeaponFourArme,WeaponFourCaliber,WeaponFourCorrectedShotDispersionMultiplier,WeaponFourDescriptorId,WeaponFourDispersionAtMaxRange,WeaponFourDispersionAtMinRange,WeaponFourEfficaciteSelonPortee,WeaponFourFX_frottement,WeaponFourFX_tir_sans_physic,WeaponFourFX_tir_tendu,WeaponFourFX_vitesse_de_depart,WeaponFourFireDescriptor,WeaponFourFireTriggeringProbability,WeaponFourGuidance,WeaponFourHitRollRule,WeaponFourIgnoreInflammabilityConditions,WeaponFourInterdireTirReflexe,WeaponFourInterfaceWeaponTexture,WeaponFourIsFireAndForget,WeaponFourIsSubAmmunition,WeaponFourLevel,WeaponFourMissileDescriptor,WeaponFourMissileTimeBetweenCorrections,WeaponFourName,WeaponFourNbTirParSalves,WeaponFourNbrProjectilesSimultanes,WeaponFourNeedModelChange,WeaponFourNoiseDissimulationMalus,WeaponFourPhysicalDamages,WeaponFourPorteeMaximale,WeaponFourPorteeMaximaleBateaux,WeaponFourPorteeMaximaleHA,WeaponFourPorteeMaximaleProjectile,WeaponFourPorteeMaximaleTBA,WeaponFourPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponFourPorteeMinimaleHA,WeaponFourPorteeMinimaleProjectile,WeaponFourPorteeMinimaleTBA,WeaponFourProjectileType,WeaponFourPuissance,WeaponFourRadiusSplashPhysicalDamages,WeaponFourRadiusSplashSuppressDamages,WeaponFourRandomDispersion,WeaponFourRayonPinned,WeaponFourSmokeDescriptor,WeaponFourSupplyCost,WeaponFourSuppressDamages,WeaponFourTempsAnimation,WeaponFourTempsDeVisee,WeaponFourTempsEntreDeuxFx,WeaponFourTempsEntreDeuxSalves,WeaponFourTempsEntreDeuxTirs,WeaponFourTirIndirect,WeaponFourTirReflexe,WeaponFourTypeArme,WeaponFourTypeName,WeaponFourWeaponCursorType,WeaponManagerID,WeaponManagerModuleSelectorID,WeaponOneAffecteParNombre,WeaponOneAffichageMenu,WeaponOneAffichageMunitionParSalve,WeaponOneAmmunitionID,WeaponOneAngleDispersion,WeaponOneAnimateOnlyOneSoldier,WeaponOneArme,WeaponOneCaliber,Wea

In [59]:
df[pd.notnull(df['WeaponFourDescriptorId'])]

,AcknowUnitType,AliasName,ArmorDescriptorFrontID,ArmorDescriptorRearID,ArmorDescriptorSidesID,ArmorDescriptorTopID,AutoOrientation,BlindagePropertiesID,CanDeploySmoke,CanWinExperience,Category,ClassNameForDebug,CommonDamageDescriptorID,CoutEtoile,CyclicManoeuvrability,DamageModuleID,DamageSelectorID,DeathExplosionAmmo,DescriptorId,DetectionTBA,EliteDeployableAmount,ExperienceGainBySecond,ExperienceModuleID,ExperienceSelectorID,Factory,FlyingAltitude,FrontArmor,FuelCapacity,FuelModuleSelectorID,FuelMoveDuration,GFactorLimit,GunMuzzleSpeed,HardenedDeployableAmount,HitRollECMModifier,HitRollSizeModifier,IconeType,IsPrototype,IsTargetableAsBoat,Key,KillExperienceBonus,LateralSpeed,ManageUnitOrientation,Mass,MaxAcceleration,MaxDamages,MaxDeceleration,MaxHPForHUD,MaxInclination,MaxPacks,MaxSuppressionDamage,Maxspeed,MinimalAltitude,MotherCountry,MovementModuleID,MovementSelectorID,MovementType,NameInMenuToken,Nationalite,OpticalStrength,OpticalStrengthAltitude,OpticalStrengthAntiradar,PaliersPhysicalDamage,PaliersSuppressDamage,PorteeVision,PorteeVisionTBA,PositionInMenu,ProductionPrice,ProductionTime,ProductionYear,RearArmor,RookieDeployableAmount,RotorArea,SalvoWeaponOneIsMain,SalvoWeaponThreeIsMain,SalvoWeaponTwoIsMain,SalvosWeaponOne,SalvosWeaponThree,SalvosWeaponTwo,SidesArmor,SpecializedDetectionPlanes,SpecializedDetectionShips,SpecializedOpticalStrengthShips,SpeedBonusOnRoad,StickToGround,StunDamageRegen,StunDamageToGetStunned,SuppressDamageRegenRatioOutOfRange,SuppressDamagesRegenRatio,TempsDemiTour,TerrainsToIgnoreMask,TextureForInterface,TextureMotherCountryForInterface,TextureTransportForInterface,TopArmor,TorqueManoeuvrability,TrainedDeployableAmount,Transporter,TurretNoneID,TurretNoneType,TurretOneID,TurretOneType,TurretThreeID,TurretThreeType,TurretTwoID,TurretTwoType,TypeForAcknow,UnitDescriptorID,UnitMovingType,UnitStealthBonus,UpgradeRequire,UpwardsSpeed,VehicleSubType,VeteranDeployableAmount,VisibilityModuleID,VisibilitySelectorID,VisionModuleID,VisionSelectorID,VisionUnitType,VitesseCombat,WeaponFourAffecteParNombre,WeaponFourAffichageMenu,WeaponFourAffichageMunitionParSalve,WeaponFourAmmunitionID,WeaponFourAngleDispersion,WeaponFourArme,WeaponFourCaliber,WeaponFourCorrectedShotDispersionMultiplier,WeaponFourDescriptorId,WeaponFourDispersionAtMaxRange,WeaponFourDispersionAtMinRange,WeaponFourEfficaciteSelonPortee,WeaponFourFX_frottement,WeaponFourFX_tir_sans_physic,WeaponFourFX_tir_tendu,WeaponFourFX_vitesse_de_depart,WeaponFourFireDescriptor,WeaponFourFireTriggeringProbability,WeaponFourGuidance,WeaponFourHitRollRule,WeaponFourIgnoreInflammabilityConditions,WeaponFourInterdireTirReflexe,WeaponFourInterfaceWeaponTexture,WeaponFourIsFireAndForget,WeaponFourIsSubAmmunition,WeaponFourLevel,WeaponFourMissileDescriptor,WeaponFourMissileTimeBetweenCorrections,WeaponFourName,WeaponFourNbTirParSalves,WeaponFourNbrProjectilesSimultanes,WeaponFourNeedModelChange,WeaponFourNoiseDissimulationMalus,WeaponFourPhysicalDamages,WeaponFourPorteeMaximale,WeaponFourPorteeMaximaleBateaux,WeaponFourPorteeMaximaleHA,WeaponFourPorteeMaximaleProjectile,WeaponFourPorteeMaximaleTBA,WeaponFourPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponFourPorteeMinimaleHA,WeaponFourPorteeMinimaleProjectile,WeaponFourPorteeMinimaleTBA,WeaponFourProjectileType,WeaponFourPuissance,WeaponFourRadiusSplashPhysicalDamages,WeaponFourRadiusSplashSuppressDamages,WeaponFourRandomDispersion,WeaponFourRayonPinned,WeaponFourSmokeDescriptor,WeaponFourSupplyCost,WeaponFourSuppressDamages,WeaponFourTempsAnimation,WeaponFourTempsDeVisee,WeaponFourTempsEntreDeuxFx,WeaponFourTempsEntreDeuxSalves,WeaponFourTempsEntreDeuxTirs,WeaponFourTirIndirect,WeaponFourTirReflexe,WeaponFourTypeArme,WeaponFourTypeName,WeaponFourWeaponCursorType,WeaponManagerID,WeaponManagerModuleSelectorID,WeaponOneAffecteParNombre,WeaponOneAffichageMenu,WeaponOneAffichageMunitionParSalve,WeaponOneAmmunitionID,WeaponOneAngleDispersion,WeaponOneAnimateOnlyOneSoldier,WeaponOneArme,WeaponOneCaliber,Wea

In [61]:
df.sample(1)

,AcknowUnitType,AliasName,ArmorDescriptorFrontID,ArmorDescriptorRearID,ArmorDescriptorSidesID,ArmorDescriptorTopID,AutoOrientation,BlindagePropertiesID,CanDeploySmoke,CanWinExperience,Category,ClassNameForDebug,CommonDamageDescriptorID,CoutEtoile,CyclicManoeuvrability,DamageModuleID,DamageSelectorID,DeathExplosionAmmo,DescriptorId,DetectionTBA,EliteDeployableAmount,ExperienceGainBySecond,ExperienceModuleID,ExperienceSelectorID,Factory,FlyingAltitude,FrontArmor,FuelCapacity,FuelModuleSelectorID,FuelMoveDuration,GFactorLimit,GunMuzzleSpeed,HardenedDeployableAmount,HitRollECMModifier,HitRollSizeModifier,IconeType,IsPrototype,IsTargetableAsBoat,Key,KillExperienceBonus,LateralSpeed,ManageUnitOrientation,Mass,MaxAcceleration,MaxDamages,MaxDeceleration,MaxHPForHUD,MaxInclination,MaxPacks,MaxSuppressionDamage,Maxspeed,MinimalAltitude,MotherCountry,MovementModuleID,MovementSelectorID,MovementType,NameInMenuToken,Nationalite,OpticalStrength,OpticalStrengthAltitude,OpticalStrengthAntiradar,PaliersPhysicalDamage,PaliersSuppressDamage,PorteeVision,PorteeVisionTBA,PositionInMenu,ProductionPrice,ProductionTime,ProductionYear,RearArmor,RookieDeployableAmount,RotorArea,SalvoWeaponOneIsMain,SalvoWeaponThreeIsMain,SalvoWeaponTwoIsMain,SalvosWeaponOne,SalvosWeaponThree,SalvosWeaponTwo,SidesArmor,SpecializedDetectionPlanes,SpecializedDetectionShips,SpecializedOpticalStrengthShips,SpeedBonusOnRoad,StickToGround,StunDamageRegen,StunDamageToGetStunned,SuppressDamageRegenRatioOutOfRange,SuppressDamagesRegenRatio,TempsDemiTour,TerrainsToIgnoreMask,TextureForInterface,TextureMotherCountryForInterface,TextureTransportForInterface,TopArmor,TorqueManoeuvrability,TrainedDeployableAmount,Transporter,TurretNoneID,TurretNoneType,TurretOneID,TurretOneType,TurretThreeID,TurretThreeType,TurretTwoID,TurretTwoType,TypeForAcknow,UnitDescriptorID,UnitMovingType,UnitStealthBonus,UpgradeRequire,UpwardsSpeed,VehicleSubType,VeteranDeployableAmount,VisibilityModuleID,VisibilitySelectorID,VisionModuleID,VisionSelectorID,VisionUnitType,VitesseCombat,WeaponFourAffecteParNombre,WeaponFourAffichageMenu,WeaponFourAffichageMunitionParSalve,WeaponFourAmmunitionID,WeaponFourAngleDispersion,WeaponFourArme,WeaponFourCaliber,WeaponFourCorrectedShotDispersionMultiplier,WeaponFourDescriptorId,WeaponFourDispersionAtMaxRange,WeaponFourDispersionAtMinRange,WeaponFourEfficaciteSelonPortee,WeaponFourFX_frottement,WeaponFourFX_tir_sans_physic,WeaponFourFX_tir_tendu,WeaponFourFX_vitesse_de_depart,WeaponFourFireDescriptor,WeaponFourFireTriggeringProbability,WeaponFourGuidance,WeaponFourHitRollRule,WeaponFourIgnoreInflammabilityConditions,WeaponFourInterdireTirReflexe,WeaponFourInterfaceWeaponTexture,WeaponFourIsFireAndForget,WeaponFourIsSubAmmunition,WeaponFourLevel,WeaponFourMissileDescriptor,WeaponFourMissileTimeBetweenCorrections,WeaponFourName,WeaponFourNbTirParSalves,WeaponFourNbrProjectilesSimultanes,WeaponFourNeedModelChange,WeaponFourNoiseDissimulationMalus,WeaponFourPhysicalDamages,WeaponFourPorteeMaximale,WeaponFourPorteeMaximaleBateaux,WeaponFourPorteeMaximaleHA,WeaponFourPorteeMaximaleProjectile,WeaponFourPorteeMaximaleTBA,WeaponFourPorteeMinimale,WeaponFourPorteeMinimaleBateaux,WeaponFourPorteeMinimaleHA,WeaponFourPorteeMinimaleProjectile,WeaponFourPorteeMinimaleTBA,WeaponFourProjectileType,WeaponFourPuissance,WeaponFourRadiusSplashPhysicalDamages,WeaponFourRadiusSplashSuppressDamages,WeaponFourRandomDispersion,WeaponFourRayonPinned,WeaponFourSmokeDescriptor,WeaponFourSupplyCost,WeaponFourSuppressDamages,WeaponFourTempsAnimation,WeaponFourTempsDeVisee,WeaponFourTempsEntreDeuxFx,WeaponFourTempsEntreDeuxSalves,WeaponFourTempsEntreDeuxTirs,WeaponFourTirIndirect,WeaponFourTirReflexe,WeaponFourTypeArme,WeaponFourTypeName,WeaponFourWeaponCursorType,WeaponManagerID,WeaponManagerModuleSelectorID,WeaponOneAffecteParNombre,WeaponOneAffichageMenu,WeaponOneAffichageMunitionParSalve,WeaponOneAmmunitionID,WeaponOneAngleDispersion,WeaponOneAnimateOnlyOneSoldier,WeaponOneArme,WeaponOneCaliber,Wea

In [58]:
df.to_csv("../data/510049986/data.csv")